# Data pre-processing

This data-set consists of bee images and a .csv file that contains data about these bees. In order to feed it into a ML model, we need to pre-process the data in order to label the data and classify this data into training, validation and/or testing data. 
Let's go ahead and work with the data:

In [100]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

In [101]:
BEE_IMAGES_PATH = 'bee_imgs'
BEE_IMAGES_REFINED_PATH = 'bee_dataset_processed'
TRAIN_FOLDER = BEE_IMAGES_REFINED_PATH + '/' + 'train'
VALID_FOLDER = BEE_IMAGES_REFINED_PATH + '/' + 'val'

In [102]:
df = pd.read_csv('bee_data.csv')

In [103]:
df.head()

,file,date,time,location,zip code,subspecies,health,pollen_carrying,caste
0,041_066.png,8/28/18,16:07,"Alvin, TX, USA",77511,-1,hive being robbed,False,worker
1,041_072.png,8/28/18,16:07,"Alvin, TX, USA",77511,-1,hive being robbed,False,worker
2,041_073.png,8/28/18,16:07,"Alvin, TX, USA",77511,-1,hive being robbed,False,worker
3,041_067.png,8/28/18,16:07,"Alvin, TX, USA",77511,-1,hive being robbed,False,worker
4,041_059.png,8/28/18,16:07,"Alvin, TX, USA",77511,-1,hive being robbed,False,worker


In [104]:
df.describe()

,zip code
count,5172.000000
mean,68599.910480
std,27101.954146
min,3431.000000
25%,50315.000000
50%,77511.000000
75%,95070.000000
max,95124.000000


In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5172 entries, 0 to 5171
Data columns (total 9 columns):
file               5172 non-null object
date               5172 non-null object
time               5172 non-null object
location           5172 non-null object
zip code           5172 non-null int64
subspecies         5172 non-null object
health             5172 non-null object
pollen_carrying    5172 non-null bool
caste              5172 non-null object
dtypes: bool(1), int64(1), object(7)
memory usage: 328.4+ KB


In [106]:
df.isnull().all()

file               False
date               False
time               False
location           False
zip code           False
subspecies         False
health             False
pollen_carrying    False
caste              False
dtype: bool

Let's see if the number of images is equal to the number of species in the DataFrame:

In [107]:
len(os.listdir(BEE_IMAGES_PATH)) == len(df['subspecies'])

True

Now that we know that there is no missing data in the csv and the all the images have been classified properly, let's go ahead and extract the useful columns from the DataFrame. We only need the file and the subspecies column:

In [108]:
df = df[['file','subspecies']]

In [109]:
df.head()

,file,subspecies
0,041_066.png,-1
1,041_072.png,-1
2,041_073.png,-1
3,041_067.png,-1
4,041_059.png,-1


We can see that certain subspecies are classified as '-1', we need to gather more information about this column:

In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5172 entries, 0 to 5171
Data columns (total 2 columns):
file          5172 non-null object
subspecies    5172 non-null object
dtypes: object(2)
memory usage: 80.9+ KB


In [111]:
df.subspecies.describe()

count                  5172
unique                    7
top       Italian honey bee
freq                   3008
Name: subspecies, dtype: object

In [112]:
df.subspecies.value_counts()

Italian honey bee        3008
Russian honey bee         527
Carniolan honey bee       501
1 Mixed local stock 2     472
-1                        428
VSH Italian honey bee     199
Western honey bee          37
Name: subspecies, dtype: int64

We can see that a couple of names are not very clean. One of the values is '-1', which means the data is not available yet, let's go ahead and rename that to 'Unknown'. Moreover, the column named '1 Mixed local stock 2' must be renamed to something that looks better. We will replace these values as follows:

In [113]:
df.subspecies.replace('-1','Unknown', inplace=True)
df.subspecies.replace('1 Mixed local stock 2', 'Mixed local stock', inplace=True)

In [114]:
df.subspecies.value_counts()

Italian honey bee        3008
Russian honey bee         527
Carniolan honey bee       501
Mixed local stock         472
Unknown                   428
VSH Italian honey bee     199
Western honey bee          37
Name: subspecies, dtype: int64

In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5172 entries, 0 to 5171
Data columns (total 2 columns):
file          5172 non-null object
subspecies    5172 non-null object
dtypes: object(2)
memory usage: 80.9+ KB


Let us randomly shuffle the data because the data is ordered in the DataFrame:

In [116]:
df = df.sample(frac=1)
df.head()

,file,subspecies
3063,043_040.png,Unknown
2643,004_485.png,Italian honey bee
5127,033_066.png,Western honey bee
4194,038_388.png,Mixed local stock
1454,005_431.png,Italian honey bee


We now have the proper subspecies names and the file where these images are located, let us go ahead and process this data to divide it into training and testing folders. We will split the DataFrame into 2 seperate DataFrames, one for training and one for validation. We will split it in the ratio '80:20':

In [117]:
len(df)

5172

In [118]:
train_valid_split = int(len(df) * (0.8))
train_valid_split

4137

In [119]:
train_df = df[:train_test_split+1]
val_df = df[train_test_split+1:]

In [120]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4138 entries, 3063 to 3419
Data columns (total 2 columns):
file          4138 non-null object
subspecies    4138 non-null object
dtypes: object(2)
memory usage: 97.0+ KB


In [121]:
val_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1034 entries, 4919 to 2939
Data columns (total 2 columns):
file          1034 non-null object
subspecies    1034 non-null object
dtypes: object(2)
memory usage: 24.2+ KB


In [122]:
train_df.tail()

,file,subspecies
1285,005_510.png,Italian honey bee
716,008_302.png,VSH Italian honey bee
4155,032_901.png,Russian honey bee
1158,030_794.png,Italian honey bee
3419,015_945.png,Italian honey bee


In [123]:
val_df.head()

,file,subspecies
4919,019_1164.png,Italian honey bee
882,030_360.png,Italian honey bee
126,017_107.png,Italian honey bee
5113,014_002.png,Unknown
5142,033_075.png,Western honey bee


Now that are training/validation dataframes are ready with the split, we can go ahead and extract labels in order to divide the randomly distributed images into their specific folders:

In [136]:
labels = list(df['subspecies'].unique())
labels

['Unknown',
 'Italian honey bee',
 'Western honey bee',
 'Mixed local stock',
 'VSH Italian honey bee',
 'Carniolan honey bee',
 'Russian honey bee']

Now, as we have extracted the unique labels, we will go and define a couple of functions to go ahead and make sub-directories of these classes in the specific sub-folders, ie. train and val. 

In [125]:
def create_train_folders(train_folder_path, labels):
    for label in labels:
        FOLDER_NAME = train_folder_path + '/' + label
        os.mkdir(FOLDER_NAME)
        
def create_val_folders(val_folder_path, labels):
    for label in labels:
        FOLDER_NAME = val_folder_path + '/' + label
        os.mkdir(FOLDER_NAME)

In [126]:
create_train_folders(TRAIN_FOLDER, labels)
create_val_folders(VALID_FOLDER, labels)

We can see that the sub-folders have been created in the respective directories as below:

In [137]:
os.listdir(TRAIN_FOLDER)

['Carniolan honey bee',
 'Italian honey bee',
 'Mixed local stock',
 'Russian honey bee',
 'Unknown',
 'VSH Italian honey bee',
 'Western honey bee']

In [138]:
os.listdir(VALID_FOLDER)

['Carniolan honey bee',
 'Italian honey bee',
 'Mixed local stock',
 'Russian honey bee',
 'Unknown',
 'VSH Italian honey bee',
 'Western honey bee']

Now, we will define functions that will allow us to read the file from the .csv into the notebook and then save it in the respective subfolder class in the training/validation folder. This will be the last step for extracting data and processing it into the appropriate place. We can see the functions as follows:

In [139]:
def divide_train_batch(image_path, train_folder_path, train_df):
    for index, row in train_df.iterrows():
        filename = row['file']
        file_path = image_path + '/' + filename
        image = cv2.imread(file_path)
        label = row['subspecies']
        train_subfolder = train_folder_path + '/' + label + '/'
        cv2.imwrite(os.path.join(train_subfolder,filename),image)
        
def divide_val_batch(image_path, val_folder_path, val_df):
    for index, row in val_df.iterrows():
        filename = row['file']
        file_path = image_path + '/' + filename
        image = cv2.imread(file_path)
        label = row['subspecies']
        val_subfolder = val_folder_path + '/' + label + '/'
        cv2.imwrite(os.path.join(val_subfolder,filename),image)

In [135]:
divide_train_batch(BEE_IMAGES_PATH, TRAIN_FOLDER, train_df)
divide_val_batch(BEE_IMAGES_PATH, VALID_FOLDER, val_df)

043_040.png bee_imgs/043_040.png Unknown bee_dataset_processed/train/Unknown/
004_485.png bee_imgs/004_485.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
033_066.png bee_imgs/033_066.png Western honey bee bee_dataset_processed/train/Western honey bee/
038_388.png bee_imgs/038_388.png Mixed local stock bee_dataset_processed/train/Mixed local stock/
005_431.png bee_imgs/005_431.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
008_178.png bee_imgs/008_178.png VSH Italian honey bee bee_dataset_processed/train/VSH Italian honey bee/
021_158.png bee_imgs/021_158.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
022_162.png bee_imgs/022_162.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
040_043.png bee_imgs/040_043.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
039_005.png bee_imgs/039_005.png Unknown bee_dataset_processed/train/Unknown/
004_472.png bee_imgs/004_472.png Italian honey bee b

007_159.png bee_imgs/007_159.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
004_211.png bee_imgs/004_211.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
038_415.png bee_imgs/038_415.png Mixed local stock bee_dataset_processed/train/Mixed local stock/
037_010.png bee_imgs/037_010.png Unknown bee_dataset_processed/train/Unknown/
022_390.png bee_imgs/022_390.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
038_043.png bee_imgs/038_043.png Mixed local stock bee_dataset_processed/train/Mixed local stock/
038_327.png bee_imgs/038_327.png Mixed local stock bee_dataset_processed/train/Mixed local stock/
015_821.png bee_imgs/015_821.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
030_735.png bee_imgs/030_735.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
024_013.png bee_imgs/024_013.png Unknown bee_dataset_processed/train/Unknown/
015_943.png bee_imgs/015_943.png Italian honey bee bee_datas

015_1050.png bee_imgs/015_1050.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
003_267.png bee_imgs/003_267.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
032_799.png bee_imgs/032_799.png Russian honey bee bee_dataset_processed/train/Russian honey bee/
022_425.png bee_imgs/022_425.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
040_498.png bee_imgs/040_498.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
005_416.png bee_imgs/005_416.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
010_627.png bee_imgs/010_627.png Carniolan honey bee bee_dataset_processed/train/Carniolan honey bee/
038_230.png bee_imgs/038_230.png Mixed local stock bee_dataset_processed/train/Mixed local stock/
030_213.png bee_imgs/030_213.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
040_228.png bee_imgs/040_228.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
005_602.png be

028_021.png bee_imgs/028_021.png Unknown bee_dataset_processed/train/Unknown/
030_724.png bee_imgs/030_724.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
046_167.png bee_imgs/046_167.png Carniolan honey bee bee_dataset_processed/train/Carniolan honey bee/
008_212.png bee_imgs/008_212.png VSH Italian honey bee bee_dataset_processed/train/VSH Italian honey bee/
012_013.png bee_imgs/012_013.png Unknown bee_dataset_processed/train/Unknown/
010_799.png bee_imgs/010_799.png Carniolan honey bee bee_dataset_processed/train/Carniolan honey bee/
040_388.png bee_imgs/040_388.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
004_203.png bee_imgs/004_203.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
030_526.png bee_imgs/030_526.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
016_289.png bee_imgs/016_289.png Unknown bee_dataset_processed/train/Unknown/
010_572.png bee_imgs/010_572.png Carniolan honey bee bee_dataset

005_434.png bee_imgs/005_434.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
005_499.png bee_imgs/005_499.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
032_702.png bee_imgs/032_702.png Russian honey bee bee_dataset_processed/train/Russian honey bee/
032_774.png bee_imgs/032_774.png Russian honey bee bee_dataset_processed/train/Russian honey bee/
016_205.png bee_imgs/016_205.png Unknown bee_dataset_processed/train/Unknown/
040_070.png bee_imgs/040_070.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
015_1156.png bee_imgs/015_1156.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
030_400.png bee_imgs/030_400.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
005_274.png bee_imgs/005_274.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
037_017.png bee_imgs/037_017.png Unknown bee_dataset_processed/train/Unknown/
009_083.png bee_imgs/009_083.png Italian honey bee bee_dat

...

036_042.png bee_imgs/036_042.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
008_272.png bee_imgs/008_272.png VSH Italian honey bee bee_dataset_processed/train/VSH Italian honey bee/
005_605.png bee_imgs/005_605.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
017_121.png bee_imgs/017_121.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
004_415.png bee_imgs/004_415.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
005_264.png bee_imgs/005_264.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
046_094.png bee_imgs/046_094.png Carniolan honey bee bee_dataset_processed/train/Carniolan honey bee/
015_562.png bee_imgs/015_562.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
022_344.png bee_imgs/022_344.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
019_1178.png bee_imgs/019_1178.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
001_09

038_361.png bee_imgs/038_361.png Mixed local stock bee_dataset_processed/train/Mixed local stock/
009_052.png bee_imgs/009_052.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
040_125.png bee_imgs/040_125.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
035_056.png bee_imgs/035_056.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
040_550.png bee_imgs/040_550.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
029_012.png bee_imgs/029_012.png Unknown bee_dataset_processed/train/Unknown/
031_116.png bee_imgs/031_116.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
015_1228.png bee_imgs/015_1228.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
015_1016.png bee_imgs/015_1016.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
038_360.png bee_imgs/038_360.png Mixed local stock bee_dataset_processed/train/Mixed local stock/
008_086.png bee_imgs/008_086.png VSH

017_125.png bee_imgs/017_125.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
004_075.png bee_imgs/004_075.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
010_783.png bee_imgs/010_783.png Carniolan honey bee bee_dataset_processed/train/Carniolan honey bee/
041_050.png bee_imgs/041_050.png Unknown bee_dataset_processed/train/Unknown/
010_593.png bee_imgs/010_593.png Carniolan honey bee bee_dataset_processed/train/Carniolan honey bee/
010_669.png bee_imgs/010_669.png Carniolan honey bee bee_dataset_processed/train/Carniolan honey bee/
010_342.png bee_imgs/010_342.png Carniolan honey bee bee_dataset_processed/train/Carniolan honey bee/
010_699.png bee_imgs/010_699.png Carniolan honey bee bee_dataset_processed/train/Carniolan honey bee/
037_018.png bee_imgs/037_018.png Unknown bee_dataset_processed/train/Unknown/
030_796.png bee_imgs/030_796.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
016_292.png bee_imgs/016_292.png Unknown

033_090.png bee_imgs/033_090.png Western honey bee bee_dataset_processed/train/Western honey bee/
008_229.png bee_imgs/008_229.png VSH Italian honey bee bee_dataset_processed/train/VSH Italian honey bee/
015_822.png bee_imgs/015_822.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
022_415.png bee_imgs/022_415.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
001_095.png bee_imgs/001_095.png Russian honey bee bee_dataset_processed/train/Russian honey bee/
030_868.png bee_imgs/030_868.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
004_484.png bee_imgs/004_484.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
005_391.png bee_imgs/005_391.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
008_246.png bee_imgs/008_246.png VSH Italian honey bee bee_dataset_processed/train/VSH Italian honey bee/
015_998.png bee_imgs/015_998.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
038_

033_084.png bee_imgs/033_084.png Western honey bee bee_dataset_processed/train/Western honey bee/
005_510.png bee_imgs/005_510.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
008_302.png bee_imgs/008_302.png VSH Italian honey bee bee_dataset_processed/train/VSH Italian honey bee/
032_901.png bee_imgs/032_901.png Russian honey bee bee_dataset_processed/train/Russian honey bee/
030_794.png bee_imgs/030_794.png Italian honey bee bee_dataset_processed/train/Italian honey bee/
015_945.png bee_imgs/015_945.png Italian honey bee bee_dataset_processed/train/Italian honey bee/


We can see that the directories and subdirectories containing the images in their respective classes have been created:

In [145]:
os.listdir(BEE_IMAGES_REFINED_PATH)

['train', 'val']

Let us see the images of the subspecies 'Italian honey bee' in the training and validation sub-directories:

In [146]:
os.listdir(os.path.join(TRAIN_FOLDER,labels[labels.index('Italian honey bee')]))

['003_009.png',
 '003_020.png',
 '003_021.png',
 '003_034.png',
 '003_035.png',
 '003_036.png',
 '003_037.png',
 '003_081.png',
 '003_094.png',
 '003_095.png',
 '003_109.png',
 '003_120.png',
 '003_121.png',
 '003_122.png',
 '003_123.png',
 '003_130.png',
 '003_134.png',
 '003_135.png',
 '003_136.png',
 '003_137.png',
 '003_143.png',
 '003_144.png',
 '003_145.png',
 '003_146.png',
 '003_147.png',
 '003_151.png',
 '003_152.png',
 '003_155.png',
 '003_156.png',
 '003_157.png',
 '003_169.png',
 '003_180.png',
 '003_181.png',
 '003_182.png',
 '003_183.png',
 '003_185.png',
 '003_186.png',
 '003_187.png',
 '003_190.png',
 '003_191.png',
 '003_192.png',
 '003_194.png',
 '003_195.png',
 '003_197.png',
 '003_209.png',
 '003_218.png',
 '003_220.png',
 '003_221.png',
 '003_222.png',
 '003_223.png',
 '003_225.png',
 '003_227.png',
 '003_230.png',
 '003_231.png',
 '003_232.png',
 '003_233.png',
 '003_235.png',
 '003_236.png',
 '003_237.png',
 '003_240.png',
 '003_241.png',
 '003_243.png',
 '003_24

In [147]:
os.listdir(os.path.join(TRAIN_FOLDER,labels[labels.index('Italian honey bee')]))

['003_009.png',
 '003_020.png',
 '003_021.png',
 '003_034.png',
 '003_035.png',
 '003_036.png',
 '003_037.png',
 '003_081.png',
 '003_094.png',
 '003_095.png',
 '003_109.png',
 '003_120.png',
 '003_121.png',
 '003_122.png',
 '003_123.png',
 '003_130.png',
 '003_134.png',
 '003_135.png',
 '003_136.png',
 '003_137.png',
 '003_143.png',
 '003_144.png',
 '003_145.png',
 '003_146.png',
 '003_147.png',
 '003_151.png',
 '003_152.png',
 '003_155.png',
 '003_156.png',
 '003_157.png',
 '003_169.png',
 '003_180.png',
 '003_181.png',
 '003_182.png',
 '003_183.png',
 '003_185.png',
 '003_186.png',
 '003_187.png',
 '003_190.png',
 '003_191.png',
 '003_192.png',
 '003_194.png',
 '003_195.png',
 '003_197.png',
 '003_209.png',
 '003_218.png',
 '003_220.png',
 '003_221.png',
 '003_222.png',
 '003_223.png',
 '003_225.png',
 '003_227.png',
 '003_230.png',
 '003_231.png',
 '003_232.png',
 '003_233.png',
 '003_235.png',
 '003_236.png',
 '003_237.png',
 '003_240.png',
 '003_241.png',
 '003_243.png',
 '003_24

As you can see, we have successfully cleaned the data, renamed columns to approriate names, extracted data from a .csv file, read images from a random distribution and create a structure that can be directly fed into a machine learning model such as CNN using different libraries like Keras.